In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_3161146/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_12_6,0.460250,0.533757,0.649208,0.067367,0.144792,0.276782,0.298784,0.191640,0.197389,0.380515,0.265021,0.386508,37.864917,74.829501,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1,model_1_12_5,0.460067,0.533396,0.648239,0.070130,0.144841,0.276996,0.299610,0.191072,0.197483,0.380580,0.264772,0.386573,37.864238,74.828822,"Hidden Size=[4], regularizer=0.02, learning_ra..."
2,model_1_13_2,0.463601,0.528903,0.643032,0.614239,0.143893,0.279664,0.304045,0.131333,0.197088,0.379332,0.269584,0.385306,37.877371,74.841955,"Hidden Size=[4], regularizer=0.02, learning_ra..."
3,model_1_13_6,0.464757,0.524909,0.637390,0.601119,0.143583,0.282035,0.308850,0.135799,0.197009,0.378923,0.271159,0.384891,37.881688,74.846271,"Hidden Size=[4], regularizer=0.02, learning_ra..."
4,model_1_13_9,0.465690,0.520422,0.631424,0.585804,0.143332,0.284698,0.313931,0.141013,0.196822,0.378593,0.272429,0.384555,37.885177,74.849761,"Hidden Size=[4], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,model_2_32_6,0.999282,-2.536984,0.997628,0.868116,0.101057,523.049336,0.904479,0.249894,0.046816,0.317895,0.996936,0.322901,102.584142,209.129119,"Hidden Size=[12], regularizer=0.2, learning_ra..."
615,model_2_32_9,0.999358,-2.554237,0.997580,0.867744,0.090267,525.600652,0.922778,0.250599,0.042347,0.300444,0.997263,0.305176,102.809973,209.354950,"Hidden Size=[12], regularizer=0.2, learning_ra..."
616,model_2_31_9,0.997970,-2.569688,0.996828,0.995460,0.285626,527.885577,1.209640,0.664269,0.070347,0.534440,0.991339,0.542857,100.506145,207.051121,"Hidden Size=[12], regularizer=0.2, learning_ra..."
617,model_2_32_4,0.998990,-2.577487,0.997690,0.899325,0.142049,529.038844,0.880940,0.190760,0.060667,0.376894,0.995692,0.382829,101.903169,208.448145,"Hidden Size=[12], regularizer=0.2, learning_ra..."
